In [1]:
import pandas as pd
import numpy as np
import math
# import tenseal as ts  #for DE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from keras.models import Sequential, load_model
from keras.utils import plot_model
from keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix


In [2]:
#loading the wednesday dataset which contains LDOS data

df = pd.read_csv('Final_data.csv')
# df.drop(1676519)   #because it contains a row having differnt label

In [3]:
# df1 = pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')
# # df.drop(1676519)   #because it contains a row having differnt label
# print(df1)
# # X = df.drop('Label', axis=1)
# Y = df1[' Label']

# print(Y.unique())
# value_counts = Y.value_counts()
# print(value_counts)

In [4]:
#data preprocessing

#replacing NAN values with 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Separating the features and label
X = df.drop('Label', axis=1)
Y = df['Label']

non_float_cols = X.select_dtypes(exclude='float64').columns

# Convert non-float columns to float, handling potential errors
for col in non_float_cols:
    try:
        X[col] = pd.to_numeric(df[col], errors='coerce')
        # print(col)
    except ValueError:
        print(f"Error converting column '{col}' to float. Check for non-numeric values.")

X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)

# # Standardize the features
#i dont want to do this because it contains things like ip add ports and all
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# pca for dimensionality reduction
# pca = PCA(n_components=10)
# X_pca = pca.fit_transform(X)

# print(Y.unique()) #['attack' 'normal']

# # for i in range(len(Y)):
# #   if Y[i]==0:
# #     print(i)

#replacing the labels with 0 for non attcak and 1 for LDOS

Y.replace('normal', 0, inplace=True)
Y.replace('attack', 1, inplace=True)

print(Y.unique()) # 0 1
print(X.shape)
print(Y.shape)



[0 1]
(2053559, 27)
(2053559,)


<ipython-input-4-dc19d36f2712>:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y.replace('attack', 1, inplace=True)


In [5]:
# Step 1: Split into testing set and the remaining data
X_train_remain, X_test, Y_train_remain, Y_test = train_test_split(X, Y, test_size=300000, random_state=42, stratify=Y)

print(X_train_remain.shape)

# Step 2: Create the first training set for initially training the base model
X_large_train = X_train_remain.iloc[:371852]
Y_large_train = Y_train_remain.iloc[:371852]

# Remaining data for training the federated clients
X_remaining = X_train_remain.iloc[371852:]
Y_remaining = Y_train_remain.iloc[371852:]

print(X_remaining.shape)


# Step 3: Create 15 smaller training sets, 3 clients 5 model aggregation iterations
X_small_train_sets = []
Y_small_train_sets = []

# Split remaining data into 15 parts
for i in range(15):
    X_small_train = X_remaining.iloc[i * 100000:(i + 1) * 100000]
    Y_small_train = Y_remaining.iloc[i * 100000:(i + 1) * 100000]
    X_small_train_sets.append(X_small_train)
    Y_small_train_sets.append(Y_small_train)

print(X_small_train_sets[14].shape)

# print(X_train)
# print(y_train)

#let us reshape consodering the timestamp

X_large_train_reshaped=X_large_train.to_numpy().reshape((X_large_train.shape[0],1,X_large_train.shape[1]))
X_train_reshaped = [None]*15
# math.floor((i+1+2)/3)
for i in range(15):
  # print(i)
  X_train_reshaped[i]=X_small_train_sets[i].to_numpy().reshape((X_small_train_sets[i].shape[0],1, X_small_train_sets[i].shape[1])) #row timestamp Columns, here timestamp will have the values 1,2,3,4,5 indicating the period in which we collected the data

X_test_reshaped = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))#row timestamp Columns



(1753559, 27)
(1381707, 27)
(0, 27)


In [6]:
def create_sequential():
  model = Sequential()
  model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
  model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
  model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

def print_accuracy_loss(model,X_test,y_test):
  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print("Accuracy and loss is ",accuracy, loss)

def print_confusion_matrix(model,X_test,y_test):
  y_pred = model.predict(X_test)
  y_pred_binary = (y_pred > 0.5).astype(int)
  cm = confusion_matrix(y_test, y_pred_binary)
  print("Confusion Matrix:")
  print(cm)

def plot_confusion_matrix(model,X_test, y_test, labels=['Attack', 'Normal']):
  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  cm = confusion_matrix(y_test, y_pred_classes)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot()
  plt.show()

def create_improved_sequential():
    model = Sequential()
    # First LSTM layer with Bidirectional and Dropout
    model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh', kernel_regularizer='l2'), input_shape=(X_large_train_reshaped.shape[1], X_large_train_reshaped.shape[2])))
    model.add(Dropout(0.3))
    # Second LSTM layer
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
    model.add(Dropout(0.3))
    # Dense layers with Batch Normalization
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)


In [7]:
#using Homomorphic Encryption (HE) and Differential Privacy (DP)
#in HE we send the encrypted weights and averging is performed on these encrypted weights and after averaging the weights are decrypted and then the model is updated with these weights
#in DP we add some noise to the weights before sending it so that the plain data is not sent, more noise we add more we corrupt the data and we can control it using epsilon. smaller the epsilon more noise is induced , even if epsilon is 1 some amount of noise in induced because we use laplacian

# Generate a context for HE
# context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
# context.global_scale = 2**40

#Add Noise to Weights:
def add_dp_noise(weights, epsilon=1.0):
    noise = [np.random.laplace(0, 1/epsilon, w.shape) for w in weights]
    noisy_weights = [w + n for w, n in zip(weights, noise)]
    return noisy_weights



In [8]:
#Asynchronous updation

def asynchronous_update(global_model,total_samples,remote_samples,remote_model_weight):
  #total_samples- total number of samples that have been used to train
  #remote_samples- total number of samples that have been used by the remote model to train

    for i in range(len(global_model)):
        # Compute incremental weighted average
        global_model[i] = (
            (global_model[i] * total_samples) + (remote_model_weight[i] * remote_samples)
        ) / (total_samples + remote_samples)

    return global_model

def decay(t,choice=1):
  b=0.01
  if choice==1:
    return 1/(1+math.exp(b*t))  #exponential decay
  else:
    return 1/(1+b*t) #linear decay

def staleness_asynchronous_update(global_model,total_samples,remote_samples,remote_model_weight,t):
  #here t represents how far from the global model is the remote model
  for i in range(len(global_model)):
        # Compute incremental weighted average with decay
        global_model[i] = decay(t)*global_model[i] + (1-decay(t))*((global_model[i] * total_samples) + (remote_model_weight[i] * remote_samples)
         / (total_samples + remote_samples))

  return global_model


In [10]:


#training the federated architecture

weightstemp=None;

#no of federated clients
remote_models_count=3
#no of federated iterations
federated_iterations=5
#number of epochs
epochs_count=1

#creating a 2D list to store the models of the federated clients and to store the weights of those models
remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
weights_remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]

#we will use this for DP and HE
noisy_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
encrypted_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]


#first we train the base model and then save it so that we can forward the parameters
base_model = create_sequential()
history_base = base_model.fit(X_large_train_reshaped , Y_large_train, epochs = epochs_count,validation_split=0.2, verbose = 1)
print("accuracy and loss before federated learning is ,")
# print_accuracy_loss(base_model,X_test_reshaped,Y_test)
base_model.save('base_model.h5')
global_model=base_model.get_weights()

count=0;

for i in range(federated_iterations):
  for j in range(remote_models_count):
    remote_models[i][j]=load_model('base_model.h5')
    adam = tf.keras.optimizers.Adam()  # Define your optimizer
    remote_models[i][j].compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])  # Recompile
    print(count)
    remote_models[i][j].fit(X_train_reshaped[count], Y_small_train_sets[count], epochs = epochs_count,validation_split=0.2, verbose = 1)
    print("accuracy and loss of",j, "client in",i," federated iteration is ")
    # print_accuracy_loss(remote_models[i][j],X_test_reshaped,Y_test)
    remote_models[i][j].save('remote_model'+str(i+1)+str(j+1)+'.h5')

    #saving the weights of the model to our weights list
    weights_remote_models[i][j]=remote_models[i][j].get_weights()


    # Adding noise to these weights
    noisy_weights[i][j] = add_dp_noise(weights_remote_models[i][j])


    count=count+1

  ##synchronous weighted average, we update the global model only after each federated iteration ie only after all the clients send their model
  # averaged_noisy_weights=[(w1 + w2 + w3) / 3 for w1, w2, w3 in zip(noisy_weights[i][0],noisy_weights[i][2],noisy_weights[i][2])]
  # global_model=averaged_noisy_weights
  # base_model.set_weights(global_model)


  #Asynchronous updation we update the global model as soon as any remote client sends his model
    # global_model=asynchronous_update(global_model,count+1,i+1,noisy_weights[i][j])
  #staleless Asynchronous updation we update the global model as soon as any remote client sends his model
    global_model=staleness_asynchronous_update(global_model,count+1,i+1,noisy_weights[i][j],0)
    base_model.set_weights(global_model)


9297/9297 ━━━━━━━━━━━━━━━━━━━━ 54s 5ms/step - accuracy: 0.7068 - loss: 0.6861 - val_accuracy: 0.7050 - val_loss: 0.6100


accuracy and loss before federated learning is ,
0
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.7102 - loss: 0.6060 - val_accuracy: 0.7045 - val_loss: 0.6113


accuracy and loss of 0 client in 0  federated iteration is 
1
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.7081 - loss: 0.6077 - val_accuracy: 0.7078 - val_loss: 0.6072


accuracy and loss of 1 client in 0  federated iteration is 
2
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.7076 - loss: 0.6080 - val_accuracy: 0.7079 - val_loss: 0.6095


accuracy and loss of 2 client in 0  federated iteration is 
3
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.7082 - loss: 0.6071 - val_accuracy: 0.7124 - val_loss: 0.6031


KeyboardInterrupt: 

In [ ]:
print("accuracy and loss after entire FL is ,")
print_accuracy_loss(base_model,X_test_reshaped,Y_test)
plot_confusion_matrix(base_model,X_test_reshaped,Y_test)
base_model.save('federated_base_model.h5')